## Initialize the population

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import math
import re
import copy
import random
import logging
from pandas.api.types import is_string_dtype
import plotly
from typing import List, Dict, Tuple, Union
from collections import defaultdict

In [2]:
# Load the input data
input_repr_dict = catalog.load('input_repr_dict')

[07/08/24 09:41:30] INFO     Loading data from input_repr_dict (PickleDataset)...               ]8;id=931635;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=451181;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [3]:
input_repr_dict.keys()

dict_keys(['J', 'M', 'compat', 'dur', 'due', 'part_id'])

In [4]:
# Unpack dictionary keys
J = input_repr_dict['J']
M = input_repr_dict['M']
compat = input_repr_dict['compat']
dur = input_repr_dict['dur']
due = input_repr_dict['due']
part_id = input_repr_dict['part_id']
part_id_full = part_id

In [5]:
# Individuals to create
n = 5

# Changeover time in minutes for Haas machine
change_over_time = 180

# Minutes in a working day
minutes_per_day = 480

# Day range
day_range = np.arange(
    minutes_per_day,
    len(J) // 5 * minutes_per_day,
    minutes_per_day,
)


In [6]:
# Helper function, find next available machine
def find_avail_m(start: int, job_idx: int, task_num: int, day_range, dur) -> int:
    """
    Finds the next available time for a machine to start a task, considering the working day duration.

    Args:
        start (int): The starting time in the schedule in minutes.
        job_idx (int): The index of the job in the job list.
        task_num (int): The task number within the job.
        day_range: The range of days in the schedule.
        dur: The duration of each task in each job.

    Returns:
        int: The next available time for the machine to start the task.
    """
    for day in day_range:
        if start < day <= start + dur[job_idx][task_num]:
            return day
    return start + dur[job_idx][task_num]


In [7]:
def init_population(n, M, J, part_id, due, change_over_time, dur, num_inds: int = None, fill_inds: bool = False):
    """
    Initializes the population of schedules. Each schedule is a list of tasks assigned to machines with start times.
    """
    if num_inds is None:
        num_inds = n

    P = []
    percentages = np.arange(10, 101, 10)

    for i in range(num_inds):
        avail_m = {m: 0 for m in M}
        product_m = {m: 0 for m in M}
        changeover_finish_time = 0
        P_j = []

        J_temp = list(range(len(J)))
        random_roll = random.random()

        if random_roll < 0.4:
            random.shuffle(J_temp)
        elif random_roll < 0.6:
            J_temp.sort(key=lambda x: part_id[x])
        elif random_roll < 0.8:
            J_temp.sort(key=lambda x: due[x], reverse=True)
        else:
            J_temp.sort(key=lambda x: (part_id[x], due[x]), reverse=True)

        while J_temp:
            # Take the index at the end of the list and remove it from the list
            job_idx = J_temp.pop()
            job = J[job_idx]
            for task in range(len(job)):
                random_roll = random.random()

                if task == 0:
                    compat_task_0 = compat[job_idx][task]
                    preferred_machines = [
                        key
                        for key in compat_task_0
                        if product_m.get(key) == part_id[job_idx] or product_m.get(key) == 0
                    ]

                    if not preferred_machines:
                        m = (
                            min(compat_task_0, key=lambda x: avail_m.get(x))
                            if random_roll < 0.7
                            else random.choice(compat_task_0)
                        )
                    else:
                        m = (
                            min(preferred_machines, key=lambda x: avail_m.get(x))
                            if random_roll < 0.7
                            else random.choice(preferred_machines)
                        )

                    start = (
                        avail_m[m]
                        if product_m[m] == 0 or part_id[job_idx] == product_m[m]
                        else avail_m[m]
                        + change_over_time
                        + max((changeover_finish_time - avail_m[m]), 0)
                    )

                    if product_m[m] != 0 and part_id[job_idx] != product_m[m]:
                        changeover_finish_time = start
                else:
                    compat_with_task = compat[job_idx][task]
                    m = (
                        min(compat_with_task, key=lambda x: avail_m.get(x))
                        if random_roll < 0.85
                        else random.choice(compat_with_task)
                    )
                    start = max(avail_m[m], P_j[-1][3] + dur[job_idx][task - 1])

                P_j.append((job_idx, job[task], m, start, dur[job_idx][task], task))
                avail_m[m] = find_avail_m(start, job_idx, task, day_range, dur)
                product_m[m] = part_id[job_idx]

        if P_j not in P:
            P.append(P_j)
        else:
            i -= 1

        # if not fill_inds and i * 100 / num_inds in percentages:
        #     print(
        #         f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {i * 100 / num_inds}% of schedules have been created."
        #     )

    if not fill_inds:
        return P
    else:
        return P


In [8]:
P = init_population(n, M, J, part_id, due, change_over_time, dur)

In [9]:
P[0]


[
    (12, 1, 1, 0, 162.0, 0),
    (12, 2, 7, 162.0, 19.8, 1),
    (12, 3, 11, 181.8, 42.0, 2),
    (12, 4, 12, 223.8, 54.0, 3),
    (12, 5, 8, 277.8, 15.0, 4),
    (12, 6, 15, 292.8, 25.2, 5),
    (12, 7, 16, 318.0, 48.0, 6),
    (28, 1, 2, 0, 360.0, 0),
    (28, 2, 9, 360.0, 19.8, 1),
    (28, 3, 11, 379.8, 48.0, 2),
    (28, 6, 15, 427.8, 25.2, 3),
    (28, 7, 17, 453.0, 48.0, 4),
    (17, 1, 3, 0, 162.0, 0),
    (17, 2, 10, 162.0, 19.8, 1),
    (17, 3, 11, 427.8, 42.0, 2),
    (17, 4, 13, 469.8, 54.0, 3),
    (17, 5, 7, 523.8, 15.0, 4),
    (17, 6, 15, 538.8, 25.2, 5),
    (17, 7, 16, 564.0, 48.0, 6),
    (2, 1, 4, 0, 162.0, 0),
    (2, 2, 8, 292.8, 19.8, 1),
    (2, 3, 11, 469.8, 42.0, 2),
    (2, 4, 14, 511.8, 54.0, 3),
    (2, 5, 10, 565.8, 15.0, 4),
    (2, 6, 15, 580.8, 25.2, 5),
    (2, 7, 17, 606.0, 48.0, 6),
    (0, 1, 5, 0, 162.0, 0),
    (0, 2, 8, 312.6, 19.8, 1),
    (0, 3, 11, 480, 42.0, 2),
    (0, 4, 12, 522.0, 54.0, 3),
    (0, 5, 8, 576.0, 15.0, 4),
    (0, 6, 15, 

## Resolve Conflicts 

In [10]:
def evaluate_population(P, due, dur, best_scores: list = None, display_scores: bool = True, on_time_bonus: int = 5000):
    """
    Evaluates the population of schedules by calculating a score for each schedule based on the completion times
    of jobs vs. the required due date.
    """
    # Calculate scores for each schedule
    scores = [
        round(
            sum(
                (
                    # Difference between due date and completion time, multiplied by urgent_multiplier if urgent
                    due[job_idx]
                    - (start_time + dur[job_idx][-1])
                    + (
                        # Bonus for completing tasks on time
                        on_time_bonus
                        if (due[job_idx] - (start_time + dur[job_idx][-1])) > 0
                        else 0
                    )
                )
                for (
                    job_idx,
                    task,
                    machine,
                    start_time,
                    job_task_dur,
                    _,
                ) in schedule
                # Only consider the completion time of the final task
                if task + 1 == max(J[job_idx])
            )
        )
        # Evaluate each schedule in the population
        for schedule in P
    ]

    if display_scores:
        print(
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Best score: {max(scores)}, "
            f"Median score: {np.median(scores)}, Worst score: {min(scores)}"
        )
        best_scores.append(max(scores))
    
    return scores


In [11]:
def find_best_schedules(P, n_e=0.1):
    scores = evaluate_population(P, due, dur, display_scores=False)
    scored_population = sorted(zip(scores, P), key=lambda x: x[0], reverse=True)
    retain_count = max(3, int(len(P) * n_e))
    P_0 = [schedule for score, schedule in scored_population[:retain_count]]
    return P_0

In [12]:
def resolve_conflict(
    P_prime: List[Tuple[int, int, int, int, int, int]], M, J, part_id_full, change_over_time, dur,
) -> (List)[Tuple[int, int, int, int, int, int]]:
    """
    This function resolves conflicts in a given schedule. If tasks are planned on the same machine at the same time,
    it finds the first available time for each task to start on the machine.

    Parameters:
    P_prime (List[Tuple[int, int, int, int, int, int]]): A list of tuples where each tuple represents a task.
    Each task is represented as (job index, task, machine, start time, duration, task number).

    Returns:
    P_prime_sorted (List[Tuple[int, int, int, int, int, int]]): A sorted list of tuples where each tuple
    represents a task. Each task is represented as (job index, job, machine, start time, duration, task number).
    """
    
    P_prime_sorted = []
    avail_m = {m: 0 for m in M}
    product_m = {m: 0 for m in M}
    changeover_finish_time = 0
    start_times = {j: 0 for j in range(len(J))}

    for job_idx in range(len(J)):
        job = J[job_idx]
        job_tasks = sorted([entry for entry in P_prime if entry[0] == job_idx], key=lambda x: x[1])

        for task_entry in job_tasks:
            _, task, m, _, part_id, task_num = task_entry

            if task_num == 0:
                start = (
                    avail_m[m]
                    if product_m[m] == 0 or part_id_full[job_idx] == product_m[m]
                    else avail_m[m]
                    + change_over_time
                    + max((changeover_finish_time - avail_m[m]), 0)
                )
                if product_m[m] != 0 and part_id_full[job_idx] != product_m[m]:
                    changeover_finish_time = start
            else:
                start = max(
                    avail_m[m],
                    start_times[job_idx] + dur[job_idx][task_num - 1],
                )
            start_times[job_idx] = start

            avail_m[m] = find_avail_m(start, job_idx, task, day_range, dur)
            product_m[m] = part_id_full[job_idx]

            P_prime_sorted.append(
                (
                    job_idx,
                    job[task_num],
                    m,
                    start,
                    dur[job_idx][task_num],
                    task_num,
                )
            )

    return P_prime_sorted


In [13]:
def offspring(P, n_c, n_e, J):
    """
    This function generates offspring for the next generation of the population. It identifies the best schedules
    in the population, after which it randomly selects each job from the first or the second schedule.
    Conflicts are resolved by another function.

    Returns:
    P_0
    """
    
    P_0 = find_best_schedules(P)

    iter_count = len(P) * (n_c + n_e)
    while len(P_0) < iter_count:
        P1, P2 = random.sample(P_0, 2)
        P_prime = [
            entry
            for job_idx in range(len(J))
            for entry in random.choice([P1, P2])
            if entry[0] == job_idx
        ]
        P_prime = resolve_conflict(P_prime, M, J, part_id_full, change_over_time, dur)
        if P_prime not in P_0:
            P_0.append(P_prime)

    return P_0


In [14]:
P_0 = offspring(P, n_c=0.3, n_e=0.1, J=J)

In [15]:
P_0


[
    [
        (23, 1, 1, 0, 150.0, 0),
        (23, 2, 7, 150.0, 19.8, 1),
        (23, 3, 11, 169.8, 42.0, 2),
        (23, 4, 12, 211.8, 54.0, 3),
        (23, 5, 8, 265.8, 15.0, 4),
        (23, 6, 15, 280.8, 25.2, 5),
        (23, 7, 16, 306.0, 48.0, 6),
        (4, 1, 3, 0, 162.0, 0),
        (4, 2, 9, 162.0, 19.8, 1),
        (4, 3, 11, 211.8, 42.0, 2),
        (4, 4, 13, 253.8, 54.0, 3),
        (4, 5, 10, 307.8, 15.0, 4),
        (4, 6, 15, 322.8, 25.2, 5),
        (4, 7, 17, 348.0, 48.0, 6),
        (21, 1, 2, 0, 162.0, 0),
        (21, 2, 7, 169.8, 19.8, 1),
        (21, 3, 11, 253.8, 42.0, 2),
        (21, 4, 14, 295.8, 54.0, 3),
        (21, 5, 9, 349.8, 15.0, 4),
        (21, 6, 15, 364.8, 25.2, 5),
        (21, 7, 17, 396.0, 48.0, 6),
        (19, 1, 4, 0, 162.0, 0),
        (19, 2, 7, 189.60000000000002, 19.8, 1),
        (19, 3, 11, 295.8, 42.0, 2),
        (19, 4, 12, 337.8, 54.0, 3),
        (19, 5, 7, 391.8, 15.0, 4),
        (19, 6, 15, 406.8, 25.2, 5),
        (